# **Chapter 6: Synchronization Tools – Practice Exercise Solutions**

## **6.1 Clock Interrupts and Disabling Interrupts**

**Question:**  
In Section 6.4, we mentioned that disabling interrupts frequently can affect the system’s clock. Explain why this can occur and how such effects can be minimized.

**Answer:**  
The system clock is updated at every **clock interrupt**. If interrupts are disabled—especially for extended periods—the system clock may lose accurate timekeeping because it misses these updates. Additionally, the scheduler relies on clock ticks to measure **time quanta** assigned to processes. At each clock interrupt, the scheduler checks if the currently running process’s time quantum has expired. If interrupts are disabled, scheduling accuracy degrades.

**Minimization:**  
Disable interrupts only for **very short periods**, ensuring the clock interrupt is not delayed significantly. Use higher-priority synchronization mechanisms (e.g., atomic instructions or spinlocks in multiprocessor systems) where possible instead of relying solely on interrupt disabling.



## **6.2 Busy Waiting**

**Question:**  
What is the meaning of the term *busy waiting*? What other kinds of waiting are there in an operating system? Can busy waiting be avoided altogether? Explain your answer.

**Answer:**  
**Busy waiting** occurs when a process repeatedly checks a condition in a tight loop without relinquishing the CPU. This wastes CPU cycles.

**Other Kinds of Waiting:**  
- **Blocking (Sleep/Wakeup):** The OS moves the waiting process to a blocked queue, freeing the CPU for other tasks. The process is later awakened when the condition is met.

**Avoidance:**  
Busy waiting can often be avoided using blocking synchronization primitives (e.g., semaphores with blocking `wait()`). However, busy waiting may still be used in certain scenarios—like **spinlocks** in multiprocessor systems—where context-switching overhead is higher than the expected wait time. Thus, it cannot be avoided *altogether* but can be minimized through careful design.



## **6.3 Spinlocks: Single vs. Multiprocessor Systems**

**Question:**  
Explain why spinlocks are not appropriate for single-processor systems yet are often used in multiprocessor systems.

**Answer:**  
- **Single-Processor:** Spinlocks are inappropriate because the condition to release the lock (set by another process) can only occur if that other process runs. If the spinning process holds the CPU, no other process can execute to release the lock, causing a **deadlock**.
- **Multiprocessor:** Other processes run on separate CPUs and can modify the lock state while one CPU spins, making short-duration spins efficient by avoiding context-switch overhead.



## **6.4 Atomicity of wait() and signal()**

**Question:**  
Show that, if the `wait()` and `signal()` semaphore operations are not executed atomically, then mutual exclusion may be violated.

**Answer:**  
Consider a semaphore `S` initialized to 1. Two processes, P1 and P2, execute `wait(S)` concurrently. If `wait()` is not atomic, both may read the value 1 simultaneously, each decrement it locally, and set `S` to 0 twice—resulting in `S = -1` instead of `S = -1` after two proper waits. This allows **both processes to enter their critical sections simultaneously**, violating mutual exclusion.

**Key Point:**  
Atomicity ensures that the test-and-decrement in `wait()` and increment in `signal()` are indivisible operations.



## **6.5 Binary Semaphore for Mutual Exclusion**

**Question:**  
Illustrate how a binary semaphore can be used to implement mutual exclusion among *n* processes.

**Answer:**  
Shared binary semaphore `mutex` initialized to 1.  
Each process `P_i` executes:

```c
do {
    wait(mutex);
    // Critical Section
    signal(mutex);
    // Remainder Section
} while (true);
```

**Explanation:**  
Only one process can pass `wait(mutex)` at a time because the semaphore acts as a lock. After exiting the critical section, `signal(mutex)` releases the lock.



## **6.6 Race Condition in a Banking System**

**Question:**  
Consider a banking system with `deposit(amount)` and `withdraw(amount)` functions. A husband calls `withdraw()` while his wife calls `deposit()` on a shared account concurrently. Describe the race condition and how to prevent it.

**Answer:**  
**Scenario:**  
Initial balance = $250.  
Husband: `withdraw(50)` → reads balance (250), calculates new balance = 200.  
Wife: `deposit(100)` → reads balance (250), calculates new balance = 350.  
If operations interleave non-atomically, one update may overwrite the other.

**Example Race:**  
1. Husband reads balance (250).  
2. Wife reads balance (250).  
3. Husband writes balance = 200.  
4. Wife writes balance = 350 → husband’s withdrawal lost.  
Final balance = $350 (incorrect; should be $300).

**Prevention:**  
Use **synchronization mechanisms** like:
- A **binary semaphore/mutex** around balance updates.
- Atomic transactions (if supported by system).
- Making `deposit()` and `withdraw()` critical sections protected by the same lock.